# Component Separation With SPT Data

### Tom Crawford

## What will we be doing in this notebook?

In the previous notebook, you created independent CMB and tSZ component maps from simulated three-band data. The simulated noise was uncorrelated between pixels and uniform across the maps, so you could use a single covariance matrix for the entire map, and that matrix was diagonal. Also, each band had the same angular resolution (beam).

Both ACT (Madhavacheril et al. 2020) and SPT (Bleem et al. 2021) have recently published and released the results of component-separation analyses, in which one of the main results were independent CMB and tSZ maps. In both of these analyses, the high-resolution ground-based data (ACT or SPT) was combined with Planck data. In this notebook, we will do a scaled-down version of the SPT+Planck component separation. We will still only compute a single $N_\mathrm{band}$-by-$N_\mathrm{band}$ covariance matrix. This is very much not the full description of the noise (including astrophysical signals we are not interested in) in the SPT+Planck maps. Luckily, this only makes the component maps less optimal; it does not make them biased.* We do have to deal with the fact that the different frequencies have different beams, though, otherwise the component maps will look very weird. We will do the easiest possible thing and smooth all the maps to roughly the same resolution.

(* This is actually a fairly subtle feature of linear-least squares solutions and applications like matched filtering and component separation. Think about why this might be true...)

## How to get the SPT data?

The single-frequency SPT maps that went into the Bleem et al. analysis are available at the NASA/LAMBDA site (https://lambda.gsfc.nasa.gov/) and from Lindsey Bleem's box server at Argonne (https://anl.app.box.com/s/5yr7z0xrgkebpzukcjvw7ve8hr4dwony). But we are going to cheat and use some maps that have already been combined with Planck data at the nearest frequency to each SPT band. We will work with one small subfield (centered at RA = 6h30m, declination = -55d). The files for this are on my personal KICP page: https://kicp.uchicago.edu/~tcrawfor/summer_school_data/.


## Import the usual stuff

In [2]:
import numpy as np
import matplotlib
import sys
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS

%matplotlib inline
import constants as cs # the constants module

from cmb_modules import * # the module of functions


## Get SPT (+Planck) maps

In [3]:
spt_band_names = ['90','150','220']
file_stubs = ['sptsz90_planck100','sptsz150_planck143','sptsz220_planck217']
spt_band_centers = np.array([95.,152.,219.])
nb = len(spt_band_names)
#spt_files = ['sptsz_ymaprun_map_ra6h30dec-55_' + bn + 'GHz.fits' for bn in spt_band_names]
spt_files = ['map_' + file_stub + '_ra6h30dec-55.fits' for file_stub in file_stubs]
hdulist_temp = fits.open(spt_files[0])
header = hdulist_temp[0].header
nx = header['NAXIS1']
ny = header['NAXIS2']
w = WCS(header,naxis=2)
spt_maps = np.zeros([nb,ny,nx])
spt_noise_maps = np.zeros([nb,ny,nx])
i = 0
c_min = -6e-4
c_max = 6e-4
X_width = 3000
Y_width = 3000
for spt_file in spt_files:
    hdulist_temp = fits.open(spt_file)
    maps_temp = hdulist_temp[0].data
    spt_maps[i,:,:] = maps_temp
    hdulist_temp = fits.open(spt_file.replace('map','dmap'))
    maps_temp = hdulist_temp[0].data
    spt_noise_maps[i,:,:] = maps_temp
    # plot the result
    plt.title(spt_band_names[i]+' GHz, signal map',fontsize=18)
    p=Plot_CMB_Map(spt_maps[i,:,:],c_min,c_max,X_width,Y_width)
    plt.title(spt_band_names[i]+' GHz, noise map',fontsize=18)
    p=Plot_CMB_Map(spt_noise_maps[i,:,:],c_min,c_max,X_width,Y_width)
    i += 1

IOError: [Errno 2] No such file or directory: 'map_sptsz90_planck100_ra6h30dec-55.fits'

### Smooth SPT maps to a common resolution

SPT beams are shaped roughly like Gaussians, and the full width at half maximum (FWHM) of those Gaussians are roughly 1.7, 1.2, and 1.1 at 90, 150, and 220 GHz, respectively. Let's smooth the maps so that they all have a final resolution of roughly 2 arcminutes. How would you do that?

#### Three hints

1. A Gaussian convolved with another Gaussian has a FWHM equal to the quadrature sum (square root of the sum of the squares) of the two individual Gaussians.
2. The "sigma" of a Gaussian is equal to the FWHM divided by $\sqrt{8 \times \ln{2}}$, or about 2.35.
3. The scipy function ndimage.gaussian_filter(map,sigma) smooths an array by a Gaussian with the input sigma (IN UNITS OF ARRAY INDICES OR MAP PIXELS).

In [ ]:
# smooth to a common resolution---but only the signal maps
from scipy import ndimage
destination_fwhm = 2.0 # arcmin
spt_fwhm = np.array([1.7,1.2,1.1]) # arcmin
# your code goes here

## noise levels in the SPT maps

There are two types of maps in the SPT files: "signal" maps and "noise" maps. The noise maps are created by making a map from half the data, making another map from half the data, and differencing the two. Recall that the white noise levels in the sim notebook were defined in units of microKelvin-arcminute (or uK-arcmin). Using the SPT noise maps (knowing that the SPT maps are in units of Kelvin), how would you measure the noise in units of uK-arcmin?

#### Warning: 

You can see from the plots above that the maps don't fill the entire array. Maybe only use a cutout around the center of the noise map to measure the noise.

In [4]:
# measure SPT noise levels
spt_white_noise_levels = np.zeros(nb)
# your code goes here

print(spt_white_noise_levels)


[ 0.  0.  0.]


### Constructing the band weights

Do just what you did for the sims, but with the SPT data.

In [5]:
# your code goes here

Now multiply the SPT frequency maps by the band weights.

In [6]:
spt_component_maps = np.zeros([2,ny,nx])

# your code goes here

# plot the result
compnames = ['CMB','tSZ']
c_mins = [-6e-4,-2e-4]
c_maxs = [6e-4,2e-4]
for i in np.arange(2):
    plt.title('reconstructed ' + compnames[i],fontsize=18)
    p=Plot_CMB_Map(spt_component_maps[i,:,:]-np.mean(spt_component_maps[i,:,:]),c_mins[i],c_maxs[i],X_width,Y_width)#,Title=str(bands[i])+' GHz')


NameError: name 'ny' is not defined

### Exercise: Find a cluster!

The tSZ component map is in units of "SZ strength" or---depending on exactly how you defined the tSZ SED, maybe even units of Compton-y parameter---so massive galaxy clusters will show up as bright spots. Do you see a very bright spot along the left edge of the map, a bit less than halfway up? Figure out what the x,y location of that bright spot is (either by zooming in and eyeballing or by finding a local maximum of the tSZ map). Use the astropy WCS utilities and the WCS information in the FITS header of the SPT map file to convert that x,y location to an RA,dec location, then search one of the astronomical databases (like NED: http://ned.ipac.caltech.edu/ for that location. Is there a famous galaxy cluster nearby?

In [7]:
from astropy import wcs

# your code goes here

print('RA: '+str(ra)+' deg')
print('dec: '+str(dec)+' deg')

NameError: name 'ra' is not defined

### Advanced question:

What's that wispy, filamentary stuff in the tSZ map?